In [ ]:
!pip -q install transformers

     |████████████████████████████████| 1.4MB 9.7MB/s 
     |████████████████████████████████| 2.9MB 34.3MB/s 
     |████████████████████████████████| 890kB 53.4MB/s 


In [ ]:
!pip -q install tweet-preprocessor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

In [ ]:
import numpy as np
import pandas as pd
import transformers
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig, BertModel
from torch.nn import CrossEntropyLoss
import re

if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
print(dev)
dev = torch.device(dev) 

cuda:0


In [ ]:
map = {'real': 0, 'fake': 1}

import preprocessor as p
import re
# p.set_options(p.OPT.EMOJI)

def clean(s):
  # s = p.clean(s)
  s = re.sub(r'[\?\.\!]+(?=[\?\.\!])', '', s)
  return s

## Data files
train_path = '/content/drive/MyDrive/sem5/NLP/data_mid/train.csv'
test_path = '/content/drive/MyDrive/sem5/NLP/data_mid/val.csv'

train_frame = pd.read_csv(train_path)
test_frame = pd.read_csv(test_path)

train_frame.append(test_frame)

train_data, train_labels = train_frame.tweet.apply(clean), train_frame.label.apply(lambda x: idx[x])
test_data, test_labels = test_frame.tweet.apply(clean), test_frame.label.apply(lambda x: idx[x])

In [ ]:
len(train_data)

6420

In [ ]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
lr = 2e-5
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
class dataset(Dataset):
    def __init__(self, tokenizer, sentences, labels ,max_len):
        self.len = len(sentences)
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        sentence = str(self.sentences[index])
        inputs = self.tokenizer.encode_plus(
            sentence,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        label = self.labels[index]
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [ ]:
train_sentences = train_data
test_sentences = test_data
training_set = dataset(tokenizer, train_sentences, train_labels,MAX_LEN)
testing_set = dataset(tokenizer, test_sentences, test_labels, MAX_LEN)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }
training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
model = transformers.BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.to(dev)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
lr = 2e-5
optimizer = transformers.AdamW(params =  model.parameters(), lr=lr)

In [ ]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(dev, dtype = torch.long)
        mask = data['mask'].to(dev, dtype = torch.long)
        targets = data['labels'].to(dev, dtype = torch.long)
        loss = model(ids, mask, labels = targets)[0]
        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')  
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
EPOCHS = 4
for epoch in range(EPOCHS):
    train(epoch)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Loss:  0.005897053051739931
Epoch: 1, Loss:  0.030827824026346207
Epoch: 2, Loss:  0.0015086798230186105
Epoch: 3, Loss:  0.015934893861413002


## **Training Summary**

In [ ]:
PATH = '/content/drive/MyDrive/sem5/NLP/midsem.sav'
## Save
torch.save(model.state_dict(), PATH)
## Load

# model = BERT(5)
# model.to(dev)
# model.load_state_dict(torch.load(PATH))

## **Performance On test Set**

In [ ]:
def valid(model, testing_loader):
    model.eval()
    predictions , true_labels = [], []
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(dev, dtype = torch.long)
            mask = data['mask'].to(dev, dtype = torch.long)
            targets = data['labels'].to(dev, dtype = torch.long)

            output = model(ids, mask, labels=targets)
            loss, logits = output[:2]
            logits = logits.detach().cpu().numpy()
            label_ids = targets.to('cpu').numpy()
            predictions.append(np.argmax(logits, axis = 1))
            true_labels.append(label_ids)

        return predictions, true_labels

In [ ]:
# To get the results on the test set. This data is not seen by the model
p,t = valid(model, testing_loader)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
from sklearn import metrics
y_true = []
y_pred = []
for batch in range(len(t)):
  for entry in range(len(t[batch])):
    y_true.append(t[batch][entry])
    y_pred.append(p[batch][entry])

# **Evaluation Metrics**

In [ ]:
print(metrics.classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1120
           1       0.99      0.96      0.97      1020

    accuracy                           0.98      2140
   macro avg       0.98      0.97      0.98      2140
weighted avg       0.98      0.98      0.98      2140



Confusion Matrix

In [ ]:
confusion_matrix = pd.DataFrame(metrics.confusion_matrix(y_true, y_pred))
confusion_matrix.columns = ['real', 'fake']
confusion_matrix.index = ['real', 'fake']
print(confusion_matrix)

      real  fake
real  1106    14
fake    39   981


In [ ]:
test_path = '/content/drive/MyDrive/sem5/NLP/data_mid/test.csv'
test_frame = pd.read_csv(test_path)

In [ ]:
test_frame

,id,tweet
0,1,Our daily update is published. States reported...
1,2,Alfalfa is the only cure for COVID-19.
2,3,President Trump Asked What He Would Do If He W...
3,4,States reported 630 deaths. We are still seein...
4,5,This is the sixth time a global health emergen...
...,...,...
2135,2136,#CoronaVirusUpdates: State-wise details of Tot...
2136,2137,Tonight 12(midnight) onwards Disaster Manageme...
2137,2138,296 new cases of #COVID19Nigeria; Plateau-85 E...
2138,2139,RT @CDCemergency: #DYK? @CDCgov’s One-Stop Sho...


In [ ]:
test_data = test_frame.tweet.apply(clean)
idx = 1
labels = []
fr = ['real', 'fake']
map = {'real': 0, 'fake': 1}
for sentence in test_data:
    sentence = sentence.lower()
    inputs = tokenizer(sentence, return_tensors="pt", max_length= 100 ,padding='max_length', truncation=True)
    id = inputs['input_ids'].to(dev)
    ret = tokenizer.decode(id[0])
    att = inputs['attention_mask'].to(dev)

    try:
      output = model(id, att)
    except:
      print(idx)
    logits = output[0]
    logits = logits.detach().cpu().numpy()
    out = np.argmax(logits)
    labels.append((idx, sentence, fr[out]))
    if 0<=idx<=5:
      print(sentence, fr[out])
    idx+=1

our daily update is published. states reported 734k tests 39k new cases and 532 deaths. current hospitalizations fell below 30k for the first time since june 22. https://t.co/wzsyme0sht real
alfalfa is the only cure for covid-19. fake
president trump asked what he would do if he were to catch the coronavirus https://t.co/3mewhusrzi #donaldtrump #coronavirus fake
states reported 630 deaths. we are still seeing a solid national decline. death reporting lags approximately 28 days from symptom onset according to cdc models that consider lags in symptoms time in hospital and the death reporting process. https://t.co/lbmcot3h9a real
this is the sixth time a global health emergency has been declared under the international health regulations but it is easily the most severe-@drtedros https://t.co/jvkc0ptett real


In [ ]:
for i in range(4):
  print(labels[i])

(1, 'our daily update is published. states reported 734k tests 39k new cases and 532 deaths. current hospitalizations fell below 30k for the first time since june 22. https://t.co/wzsyme0sht', 'real')
(2, 'alfalfa is the only cure for covid-19.', 'fake')
(3, 'president trump asked what he would do if he were to catch the coronavirus https://t.co/3mewhusrzi #donaldtrump #coronavirus', 'fake')
(4, 'states reported 630 deaths. we are still seeing a solid national decline. death reporting lags approximately 28 days from symptom onset according to cdc models that consider lags in symptoms time in hospital and the death reporting process. https://t.co/lbmcot3h9a', 'real')


In [ ]:
df = pd.DataFrame(labels)

In [ ]:
df.columns = ['id', 'tweet', 'label']

In [ ]:
df = df.drop(['tweet'], axis = 1)

In [ ]:
df.to_csv('/content/drive/MyDrive/sem5/NLP/data_mid/pred.csv', index = False)